In [3]:
#Basic importations
import numpy as np
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf


# This is mainly for deep understanding of paper Attention is all you need 
## Check it out here : https://arxiv.org/abs/1706.03762 .  
Basically is creating a transformer to be able to translate words from english to portuguese in this specific case.


In [6]:
#We use the data provided by Ted fundation in tensorflow_datasets
examples, metadata = tfds.load('ted_hrlr_translate/es_to_pt', with_info=True,
                               as_supervised=True)



train_examples, val_examples = examples['train'], examples['validation']

Dataset ted_hrlr_translate downloaded and prepared to C:\Users\MI-CUENTA\tensorflow_datasets\ted_hrlr_translate\es_to_pt\1.0.0. Subsequent calls will reuse this data.


In [48]:
#we will call data a class to not have a random global variable 
class Data():
    def __init__(self):
        #alternitavely we can ask in innit what language we want to train
        i=0
        #WE WILL GET THE DATASET IN WORKING CONDITIONS EVERY LIST(TRAIN_EXMAPLES[I]) GIVES US A (EXAMPLE 1, EXAMPLE 2)
        pt=[]
        es=[]
        #WE NEED TO DECODE IT TO UTF-8 SO IT IS PRESENTABLE AND WE ONLY WANT THE NUMPY
        for es_example, pt_example in list(train_examples):
            pt.append((pt_example.numpy()).decode('utf-8'))
            es.append((es_example.numpy()).decode('utf-8'))
        self.pt=pt
        self.es=es
        #And with this we have or data


Now we need to think about input embedding .
Because obviously computers dont understand strings, we need some kind of input. We will build an embedding from zero
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset -> will help us generate a word for word construction
But for this case we need it to be object database so Data() is not useful in this case, although we will use it later





In [57]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert

In [74]:
#Datos=Data()
# in this case lets generate a vocabulary of 5000 words
# WE USE MAP AND LAMBDA MAP MAPS ONTO ARRAY AND SIMPLY LAMBDA ES,PT: ES IS SETTING FROM BOTH ES AND PT SETTING THE RESULT TO ES
# THE RESEVED TOKENS ARE SIMPLY START FINISH AND CONDITRIONS
es_vocab = bert.bert_vocab_from_dataset(train_examples.map(lambda es,pt:es),vocab_size =5000,reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"])
pt_vocab = bert.bert_vocab_from_dataset(train_examples.map(lambda es,pt:pt),vocab_size =5000,reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"])


In [72]:

def write_vocab_file(filepath, vocab):
    with open(filepath, 'w',encoding='utf-8') as f:
        for token in vocab:
              print(token, file=f)

In [75]:
write_vocab_file('es_vocab.txt',es_vocab)
write_vocab_file('pt_vocab.txt',pt_vocab)



And with this we have already created a vocabulary. To  encapsulate everything we could put it in a class in this way.

In [77]:
class Vocabgen:
    def __init__(self,filename1,filename2,num):
        es_vocab = bert.bert_vocab_from_dataset(train_examples.map(lambda es,pt:es),vocab_size =num,reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"])
        pt_vocab = bert.bert_vocab_from_dataset(train_examples.map(lambda es,pt:pt),vocab_size =num,reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"])
        #WE CREATE THE VOCABS AND WE WRITE THEM
        write_vocab_file(filename1,es_vocab)
        write_vocab_file(filename2,pt_vocab)
        #We could make it even better so that given two languages we create the vocabs but is not necessary at the moment
        
    def write_vocab_file(filepath, vocab):
        with open(filepath, 'w',encoding='utf-8') as f:
            for token in vocab:
                  print(token, file=f)

In [78]:
Vocabgen('es_vocab.txt','pt_vocab.txt',5000)